<a href="https://colab.research.google.com/github/bigpine-choi/commercial-real-estate-analysis/blob/main/LSTM_%EC%98%88%EC%B8%A1_%EB%AA%A8%EB%8D%B8_%EC%98%A4%ED%94%BC%EC%8A%A4_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📈 LSTM 기반 투자수익률 예측 모델 (전체 데이터셋)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
# ✅ 파라미터 설정
time_steps = 8
predict_steps = 4
target_column = "투자수익률"
feature_columns = ["임대료", "공실률", "순영업소득", "투자수익률"]

In [ ]:
# ✅ 데이터 로딩
from google.colab import files
uploaded = files.upload()
df = pd.read_csv("/content/오피스_임대료_공실률_투자수익률_순영업소득_all.csv", encoding="utf-8-sig")

Saving 오피스_임대료_공실률_투자수익률_순영업소득_all.csv to 오피스_임대료_공실률_투자수익률_순영업소득_all.csv


In [ ]:
# ✅ 예측 함수 정의
def predict_for_region(region_name):
    region_df = df[df["CLS_NM"] == region_name].copy()
    region_df = region_df.sort_values("WRTTIME_DESC")
    pivot_df = region_df.pivot(index="WRTTIME_DESC", columns="지표", values="DTA_VAL")[feature_columns].dropna()
    if pivot_df.shape[0] < time_steps + predict_steps:
        raise ValueError("시계열 길이가 부족함")
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(pivot_df)
    X, y = [], []
    for i in range(len(scaled_data) - time_steps - predict_steps + 1):
        X.append(scaled_data[i:i+time_steps])
        y.append(scaled_data[i+time_steps:i+time_steps+predict_steps, 3])
    X, y = np.array(X), np.array(y)
    model = Sequential()
    model.add(LSTM(64, activation="relu", input_shape=(time_steps, X.shape[2])))
    model.add(Dense(predict_steps))
    model.compile(optimizer="adam", loss="mse")
    model.fit(X, y, epochs=100, batch_size=8, verbose=0)
    last_seq = scaled_data[-time_steps:]
    last_seq = np.expand_dims(last_seq, axis=0)
    pred_scaled = model.predict(last_seq)
    future_pred = scaler.inverse_transform(np.concatenate([np.zeros((predict_steps, 3)), pred_scaled.reshape(-1, 1)], axis=1))[:, -1]
    return pivot_df.index[-1], future_pred

In [ ]:
# ✅ 전체 지역 예측 실행
results = {}
for region in df["CLS_NM"].unique():
    try:
        last_date, forecast = predict_for_region(region)
        results[region] = forecast
        print(f"✅ {region} 완료 | 마지막 분기: {last_date} | 예측값: {np.round(forecast, 2)}")
    except Exception as e:
        print(f"⚠️ {region} 예측 실패: {e}")

⚠️ 신문로지역 예측 실패: "['순영업소득'] not in index"
⚠️ 우정국로지역 예측 실패: "['순영업소득'] not in index"
⚠️ 무교지역 예측 실패: "['순영업소득'] not in index"
⚠️ 청계지역 예측 실패: "['순영업소득'] not in index"
⚠️ 서울역지역 예측 실패: "['순영업소득'] not in index"
⚠️ 남대문지역 예측 실패: "['순영업소득'] not in index"
⚠️ 명동지역 예측 실패: "['순영업소득'] not in index"
⚠️ 기타지역 예측 실패: "['순영업소득'] not in index"
⚠️ 마포지역 예측 실패: "['순영업소득'] not in index"
⚠️ 국회앞지역 예측 실패: "['순영업소득'] not in index"
⚠️ 여의도중앙지역 예측 실패: "['순영업소득'] not in index"
⚠️ 증권거래소지역 예측 실패: "['순영업소득'] not in index"
⚠️ 영등포지역 예측 실패: "['순영업소득'] not in index"
⚠️ 방배지역 예측 실패: "['순영업소득'] not in index"
⚠️ 서초지역 예측 실패: "['순영업소득'] not in index"
⚠️ 도산로지역 예측 실패: "['순영업소득'] not in index"
⚠️ 역삼북부지역 예측 실패: "['순영업소득'] not in index"
⚠️ 선릉북부지역 예측 실패: "['순영업소득'] not in index"
⚠️ 삼성북부지역 예측 실패: "['순영업소득'] not in index"
⚠️ 역삼남부지역 예측 실패: "['순영업소득'] not in index"
⚠️ 선릉남부지역 예측 실패: "['순영업소득'] not in index"
⚠️ 삼성남부지역 예측 실패: "['순영업소득'] not in index"
⚠️ 양재지역 예측 실패: "['순영업소득'] not in index"
⚠️ 송파지역 예측 실패: "['순영업소득'] not in index"
⚠️ 충정로지역 예측 실패

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
✅ 광화문 완료 | 마지막 분기: 2024년 4분기 | 예측값: [2.65 2.56 1.74 1.09]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
✅ 동대문 완료 | 마지막 분기: 2024년 4분기 | 예측값: [2.1  2.14 1.53 1.09]


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
✅ 홍대/합정 완료 | 마지막 분기: 2024년 4분기 | 예측값: [1.67 1.61 0.94 1.09]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
✅ 남포동 완료 | 마지막 분기: 2024년 4분기 | 예측값: [0.15 0.4  0.32 0.12]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
✅ 부산역 완료 | 마지막 분기: 2024년 4분기 | 예측값: [1.25 1.43 0.97 0.74]


In [ ]:
# ✅ 결과 정리 및 저장
result_df = pd.DataFrame(results).T
result_df.columns = [f"예측_{i+1}분기후" for i in range(predict_steps)]
result_df["예측평균"] = result_df.mean(axis=1)
result_df = result_df.sort_values("예측평균", ascending=False)

# ✅ 출력
print("\n📌 향후 투자수익률 예측 결과 (상위 지역):")
print(result_df.head())

# ✅ 저장
result_df.to_csv("LSTM_예측_투자수익률_지역별_all.csv", encoding="utf-8-sig")


📌 향후 투자수익률 예측 결과 (상위 지역):
       예측_1분기후   예측_2분기후   예측_3분기후   예측_4분기후      예측평균
강남대로  3.236031  2.559053  2.534785  2.101943  2.607953
신사역   2.328967  2.392010  2.143230  2.807122  2.417832
천호    1.943969  2.487446  2.541084  2.549999  2.380625
시청    2.419248  2.861435  2.421274  1.729801  2.357939
숙명여대  2.286740  2.305648  1.756581  2.701312  2.262570
